### ----------------------------------------
#  Working with Matchfiles
### ----------------------------------------

Please check the following link for the matchfile contents and structure:
https://github.com/dirac-institute/alert_tools/tree/master/Matchfile

### FILE FORMAT
The ZTF matchfiles are stored in HDF5 format, and are readable and writable by the pytables Python package.

### GROUPS WITHIN THE FILE
The HDF5 groups within each file is named “matches”. For example, the hierarchy in a file is store.root.matches where store is the pytables file handle.


### FILE LOCATIONS
The matchfiles are stored on the ZTF systems in subdirectories of /ztf/ops/srcmatch, where the subdirectories take the form rcNN/frNNNNNN-NNNNNN similar to the reference images.

### FILENAMING CONVENTION
The matchfiles follow the naming convention of reference images, ztf_NNNNNN_zC_cNN_qN_match.pytable where the capital items are field number, filter letter, chip number and quadrant number.

In [1]:
import pandas as pd
import tables as pt
import os

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

pd.set_option('display.max_columns',100)
#pd.set_option('float_format', '{:f}'.format)

In [3]:
DATA_DIR = "/epyc/data/ztf_matchfiles/partnership/ztfweb.ipac.caltech.edu" 
root_dir = DATA_DIR

In [4]:
def find_files(root_dir):
    for dir_name, subdir_list, file_list in os.walk(root_dir, followlinks=True):
        for fname in file_list:
            if fname.endswith('.pytable'):
                yield dir_name+'/'+fname

In [5]:
print('{} has {} pytable files'.format(root_dir, len(list(find_files(root_dir)))))

/epyc/data/ztf_matchfiles/partnership/ztfweb.ipac.caltech.edu has 19018 pytable files


In [6]:
i = 19017 
fl = list(find_files(root_dir))[i]
fl

'/epyc/data/ztf_matchfiles/partnership/ztfweb.ipac.caltech.edu/rc63/fr001751-001800/ztf_001757_zr_c16_q4_match_programid2.pytable'

In [7]:
mtfl = pt.open_file(fl, 'r', root_uep='/',filters='lzo')

In [8]:
type(mtfl)

tables.file.File

In [9]:
mtfl

File(filename=/epyc/data/ztf_matchfiles/partnership/ztfweb.ipac.caltech.edu/rc63/fr001751-001800/ztf_001757_zr_c16_q4_match_programid2.pytable, title='List of Exposures', mode='r', root_uep='/', filters=Filters(complevel=4, complib='lzo', shuffle=True, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'List of Exposures'
/matches (Group) 'Matched Sources'
/matches/exposures (Table(1,), shuffle, lzo(4)) 'List of Exposures'
  description := {
  "diq": Float32Col(shape=(), dflt=0.0, pos=0),
  "expid": Int32Col(shape=(), dflt=0, pos=1),
  "fieldid": UInt32Col(shape=(), dflt=0, pos=2),
  "filterid": UInt16Col(shape=(), dflt=0, pos=3),
  "fracday": UInt64Col(shape=(), dflt=0, pos=4),
  "infobits": Int32Col(shape=(), dflt=0, pos=5),
  "ipac_pub_date": Float32Col(shape=(), dflt=0.0, pos=6),
  "limitmag": Float32Col(shape=(), dflt=0.0, pos=7),
  "magzp": Float32Col(shape=(), dflt=0.0, pos=8),
  "magzprms": Float32Col(shape=(), dflt=0.0, pos=9),
  "magzpunc": Float

In [10]:
for group in mtfl.walk_groups("/"):
    print(group)

/ (RootGroup) 'List of Exposures'
/matches (Group) 'Matched Sources'


In [11]:
for table in mtfl.walk_nodes("/","Table"):
    print(table)

/matches/exposures (Table(1,), shuffle, lzo(4)) 'List of Exposures'
/matches/sourcedata (Table(3905,), shuffle, lzo(4)) 'Field 001757 RC 63 Matched Sources Data'
/matches/sources (Table(36715,), shuffle, lzo(4)) 'Field 001757 RC 63 Matched Sources Table'
/matches/transientdata (Table(2327,), shuffle, lzo(4)) 'Field 001757 RC 63 Transients Data'
/matches/transients (Table(2327,), shuffle, lzo(4)) 'Field 001757 RC 63 Transients Table'


In [12]:
exposuresObject = mtfl.get_node("/matches", "exposures")
sourcedataObject = mtfl.get_node("/matches", "sourcedata")
sourcesObject = mtfl.get_node("/matches", "sources")
transientdataObject = mtfl.get_node("/matches", "transientdata")
transientsObject = mtfl.get_node("/matches", "transients")

# 1- Exposures

In [13]:
#exposures = exposuresObject.read()

In [14]:
pd.DataFrame.from_records(exposuresObject.read())

,diq,expid,fieldid,filterid,fracday,infobits,ipac_pub_date,limitmag,magzp,magzprms,magzpunc,nsources,nstars,ntransients,obshjd,obsmjd,programid,rcid,relphotsatmag,relphotsyserr,relphotzp,relphotzperr
0,1.512479,53018756,1757,2,20180615187569,0,99999.0,0.0,26.046591,0.028655,0.000016,3895,0,2327,2.458285e+06,58284.187569,2,63,0.0,0.0,0.0,0.0


# 2-  Sourcedata

In [15]:
pd.DataFrame.from_records(sourcedataObject.read())

,catflags,chi,dec,expid,hjd,mag,magerr,matchid,mjd,programid,psfflux,psffluxerr,psfmag,psfmagerr,ra,relphotflags,sharp,sid,snr,x,xpos,y,ypos,z
0,0,0.688,46.962817,53018756,2.458285e+06,18.050591,0.0,0,58284.187569,2,1578.434082,63.385311,18.050591,0.044,206.711117,0,0.024,3048,24.900000,-0.609642,2975.384033,-0.306766,70.127998,0.730911
1,0,0.581,46.974186,53018756,2.458285e+06,19.315590,0.0,6,58284.187569,2,492.507935,31.843882,19.315590,0.070,207.115069,0,-0.124,3054,15.470000,-0.607335,1994.438965,-0.310991,75.625999,0.731046
2,0,0.822,46.967409,53018756,2.458285e+06,16.073591,0.0,12,58284.187569,2,9758.375977,329.852112,16.073591,0.037,206.918743,0,-0.035,69,29.580000,-0.608474,2471.375000,-0.308947,78.016998,0.730966
3,0,0.606,46.976668,53018756,2.458285e+06,21.040590,0.0,13,58284.187569,2,100.584457,23.290127,21.040590,0.251,207.227372,0,-0.087,70,4.320000,-0.606696,1721.854980,-0.312166,78.680000,0.731076
4,0,0.534,46.994303,53018756,2.458285e+06,20.087591,0.0,18,58284.187569,2,241.831161,24.679008,20.087591,0.111,207.914747,0,-0.041,71,9.800000,-0.602709,53.013000,-0.319317,80.188004,0.731286
5,0,0.452,46.973862,53018756,2.458285e+06,20.629591,0.0,19,58284.187569,2,146.774918,21.913439,20.629591,0.162,207.149563,0,-0.052,3056,6.700000,-0.607152,1910.884033,-0.311358,80.462997,0.731043
6,0,0.932,46.984698,53018756,2.458285e+06,17.789591,0.0,21,58284.187569,2,2008.724243,86.399826,17.789591,0.047,207.544413,0,-0.069,73,23.250000,-0.604869,952.164978,-0.315471,81.578003,0.731172
7,0,0.832,46.968268,53018756,2.458285e+06,21.073591,0.0,22,58284.187569,2,97.517967,26.666777,21.073591,0.297,206.983739,0,0.193,3057,3.660000,-0.608114,2313.699951,-0.309632,82.271004,0.730976
8,0,0.542,46.980002,53018756,2.458285e+06,21.222591,0.0,24,58284.187569,2,85.032646,22.116991,21.222591,0.282,207.386629,0,0.034,76,3.840000,-0.605788,1335.328003,-0.313832,83.010002,0.731116
9,0,0.595,46.981477,53018756,2.458285e+06,15.607591,0.0,25,58284.187569,2,14978.041992,498.009857,15.607591,0.036,207.445895,0,0.021,3058,30.080000,-0.605447,1191.447998,-0.314449,83.592003,0.731133


# 3- Sources

In [16]:
sources_columns = pd.DataFrame.from_records(sourcesObject.read(0,0))
sources_columns_list = sources_columns.columns.tolist()
pd.DataFrame.from_records(sourcesObject.cols, columns=sources_columns_list)

,astrometricrms,bestastrometricrms,bestchisq,bestcon,bestlineartrend,bestmagrms,bestmaxmag,bestmaxslope,bestmeanmag,bestmedianabsdev,bestmedianmag,bestminmag,bestnabovemeanbystd,bestnbelowmeanbystd,bestnconsecabovemeanbystd,bestnconsecbelowmeanbystd,bestnconsecfrommeanbystd,bestnmedianbufferrange,bestnpairposslope,bestpercentiles,bestperiodsearch,bestprobnonqso,bestprobqso,bestskewness,bestsmallkurtosis,beststetsonj,beststetsonk,bestvonneumannratio,bestweightedmagrms,bestweightedmeanmag,chisq,con,dec,lineartrend,magrms,matchid,maxmag,maxslope,meanmag,medianabsdev,medianmag,minmag,nabovemeanbystd,nbelowmeanbystd,nbestobs,nconsecabovemeanbystd,nconsecbelowmeanbystd,nconsecfrommeanbystd,ngoodobs,nmedianbufferrange,nobs,npairposslope,percentiles,periodsearch,probnonqso,probqso,ra,refchi,refmag,refmagerr,refsharp,refsnr,skewness,smallkurtosis,stetsonj,stetsonk,uncalibmeanmag,vonneumannratio,weightedmagrms,weightedmeanmag,x,y,z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.962808,0.0,inf,0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,206.711130,0.0,18.050999,0.021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,-0.609642,-0.306766,0.730911
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.962083,0.0,inf,1,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,206.694279,0.0,22.712000,0.294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,-0.609741,-0.306591,0.730902
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.981810,0.0,inf,2,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.345716,0.0,22.830999,0.297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,-0.605992,-0.313388,0.731137
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.980246,0.0,inf,3,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.293340,0.0,22.681999,0.238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,-0.606296,-0.312843,0.731119
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.968248,0.0,inf,4,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,206.900994,0.0,22.688000,0.216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,0.0,-0.608560,-0.308754,0.730976
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0

# 4- Transientdata

In [17]:
pd.DataFrame.from_records(transientdataObject.read())

,catflags,chi,dec,expid,hjd,mag,magerr,matchid,mjd,programid,psfflux,psffluxerr,psfmag,psfmagerr,ra,relphotflags,sharp,sid,snr,x,xpos,y,ypos,z
0,0,0.822,46.992117,53018756,2.458285e+06,21.160591,0.0,0,58284.187569,2,90.060478,26.817337,21.160591,0.323,207.036257,0,-0.634,0,3.36,-0.607559,2182.542969,-0.310051,3.378000,0.731260
1,0,0.720,46.990859,53018756,2.458285e+06,21.312592,0.0,1,58284.187569,2,78.253021,25.607784,21.312592,0.355,206.976828,0,-0.157,1,3.06,-0.607894,2326.729004,-0.309428,1.265000,0.731245
2,0,0.561,47.009228,53018756,2.458285e+06,21.184591,0.0,2,58284.187569,2,88.067917,22.152111,21.184591,0.273,207.626938,0,-0.010,2,3.98,-0.604136,748.778015,-0.316197,2.084000,0.731464
3,0,0.681,47.003601,53018756,2.458285e+06,21.361591,0.0,3,58284.187569,2,74.799469,23.678474,21.361591,0.344,207.416403,0,-0.217,3,3.16,-0.605358,1259.738037,-0.314008,2.107000,0.731397
4,0,0.227,47.010045,53018756,2.458285e+06,20.924591,0.0,4,58284.187569,2,111.842278,17.985640,20.924591,0.175,207.672542,0,-0.039,4,6.22,-0.603875,638.153015,-0.316673,3.325000,0.731473
5,0,0.503,47.009239,53018756,2.458285e+06,21.279591,0.0,5,58284.187569,2,80.664322,21.575150,21.279591,0.290,207.644199,0,-0.145,5,3.74,-0.604041,706.947998,-0.316379,3.622000,0.731464
6,0,0.746,47.009635,53018756,2.458285e+06,20.039591,0.0,6,58284.187569,2,252.750793,28.796379,20.039591,0.124,207.677836,0,-0.056,6,8.78,-0.603851,625.375977,-0.316731,5.258000,0.731468
7,0,1.021,47.013388,53018756,2.458285e+06,20.218590,0.0,7,58284.187569,2,214.381500,32.362492,20.218590,0.164,207.840850,0,0.018,7,6.62,-0.602905,229.802994,-0.318425,6.170000,0.731513
8,0,1.144,46.986784,53018756,2.458285e+06,20.702591,0.0,8,58284.187569,2,137.298401,32.119949,20.702591,0.254,206.898819,0,0.176,8,4.27,-0.608361,2516.403076,-0.308624,6.942000,0.731196
9,0,0.774,46.980322,53018756,2.458285e+06,20.042591,0.0,9,58284.187569,2,252.039581,29.017128,20.042591,0.125,206.737397,0,-0.140,9,8.69,-0.609302,2908.614990,-0.306945,11.101000,0.731119


# 5- Transients

In [18]:
transients_columns = pd.DataFrame.from_records(transientsObject.read(0,0))
transients_columns_list = transients_columns.columns.tolist()
pd.DataFrame.from_records(transientsObject.cols, columns=transients_columns_list)

,astrometricrms,bestastrometricrms,bestchisq,bestcon,bestlineartrend,bestmagrms,bestmaxmag,bestmaxslope,bestmeanmag,bestmedianabsdev,bestmedianmag,bestminmag,bestnabovemeanbystd,bestnbelowmeanbystd,bestnconsecabovemeanbystd,bestnconsecbelowmeanbystd,bestnconsecfrommeanbystd,bestnmedianbufferrange,bestnpairposslope,bestpercentiles,bestperiodsearch,bestprobnonqso,bestprobqso,bestskewness,bestsmallkurtosis,beststetsonj,beststetsonk,bestvonneumannratio,bestweightedmagrms,bestweightedmeanmag,chisq,con,dec,lineartrend,magrms,matchid,maxmag,maxslope,meanmag,medianabsdev,medianmag,minmag,nabovemeanbystd,nbelowmeanbystd,nbestobs,nconsecabovemeanbystd,nconsecbelowmeanbystd,nconsecfrommeanbystd,ngoodobs,nmedianbufferrange,nobs,npairposslope,percentiles,periodsearch,probnonqso,probqso,ra,refchi,refmag,refmagerr,refsharp,refsnr,skewness,smallkurtosis,stetsonj,stetsonk,uncalibmeanmag,vonneumannratio,weightedmagrms,weightedmeanmag,x,y,z
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.992117,0.0,inf,0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1,0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.036257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.607559,-0.310051,0.731260
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.990859,0.0,inf,1,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1,0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,206.976828,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.607894,-0.309428,0.731245
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.009228,0.0,inf,2,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1,0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.626938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.604136,-0.316197,0.731464
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.003601,0.0,inf,3,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1,0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.416403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.605358,-0.314008,0.731397
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.010045,0.0,inf,4,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]",0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",1,0,1,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...",0.0,0.0,207.672542,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.603875,-0.316673,0.731473
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]","[0, 0, 0]",0,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...

In [22]:
pd.DataFrame.from_records(transientsObject.cols, columns=transients_columns_list)['periodsearch']

0       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
1       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
2       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
3       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
4       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
5       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
6       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
7       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
8       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
9       [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
10      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
11      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
12      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
13      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
14      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
15      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
16      [[0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0...
17      [[0.0,